In [1]:
from pathlib import Path
import pandas as pd

data_root = Path("../kkdata3")
for x in data_root.glob("*"):
    print(x)

train_target = pd.read_parquet(data_root / "label_train_target.parquet")
test_source = pd.read_parquet(data_root / "label_test_source.parquet")
meta_song = pd.read_parquet(data_root / "meta_song.parquet")

../kkdata3/predict_result.parquet
../kkdata3/label_train_source.parquet
../kkdata3/label_train_target.parquet
../kkdata3/label_test_source.parquet
../kkdata3/meta_song_titletext.parquet
../kkdata3/sample.csv
../kkdata3/meta_song_lyricist.parquet
../kkdata3/meta_song_producer.parquet
../kkdata3/meta_song.parquet
../kkdata3/meta_song_genre.parquet
../kkdata3/meta_song_composer.parquet


In [38]:
train_source = pd.read_parquet(data_root / "label_train_source.parquet")
train_source.sort_values(["session_id", "listening_order"], inplace=True)
train_target.sort_values(["session_id", "listening_order"], inplace=True)
# test_source.sort_values(["session_id", "listening_order"], inplace=True)

In [39]:
# return n+1 column song id
def getTrainData(df, n=2):
    df = df.copy()
    # gen n song id be the dataset
    for i in range(1, n + 1):
        df[f"next{i}_song_id"] = df["song_id"].shift(-i)

    # check if last song id is in the same session
    df[f"next{n}_session_id"] = df["session_id"].shift(-n)
    df = df.query(f"session_id == next{n}_session_id")

    # only get the song_id and next1_song_id, next2_song_id, next3_song_id... column
    df = df[["song_id"] + [f"next{i}_song_id" for i in range(1, n + 1)]]

    return df

In [40]:
src = getTrainData(train_source, 19)
tgt = getTrainData(train_target, 4)

src.reset_index(drop=True, inplace=True)
tgt.reset_index(drop=True, inplace=True)

src.shape[0] == tgt.shape[0]

True

In [51]:
src[src.duplicated()].sort_values("song_id").sort_index()
tgt[src.duplicated()].sort_values("song_id").sort_index()

,song_id,next1_song_id,next2_song_id,next3_song_id,next4_song_id
217,20e137c89079bade9c596d92757afa59,dfcce1f1ae5fed1c038830ea5f4e559b,e1517d538da361e7dc58f77bf187d00c,0a7be36a7ea8d7bcb6dc705361fac0ea,916bf51f5bce48fecda79154c93e1ce5
590,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7
617,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d
618,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d
619,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d
...,...,...,...,...,...
572101,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632
572142,7b4ec012f3f5580e57add7139b1c6e02,8797a41daa53cab47d7aa0d75bd7dbb4,98e159f2fe90193c82209674b638ee1c,7b4ec012f3f5580e57add7139b1c6e02,66e6cbab82630d69acca2115157fbcba
572205,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6
572233,9ad9c63b533186746f04a612abca0860,7e0caaaa8bd33ad2c6e6000458e4b2b1,767730509063e5779182491d1668aac0,9fe2262287d3f0415f6e5c8999ee8a1c,fdef56717e5431c7b1c8325222f2406a


In [54]:
duplicates_mask = src.duplicated()

(572259, 20)

In [53]:
tgt_filtered = tgt[~duplicates_mask].sort_values("song_id").sort_index()
src_filtered = src[~duplicates_mask].sort_values("song_id").sort_index()
tgt_filtered.shape

(528072, 5)

In [24]:
# show duplicated index
src[src.duplicated(keep=False)].index

Index([ 8414621,   699907,  4133579,  5432417, 11266145, 11244745, 10309440,
        1321119, 10398120,  7872844,
       ...
       10820813,  8504321,  5432397,  1007379,  9445143,  9399585,  6459189,
        8325721, 10777413,  1343539],
      dtype='int64', length=50852)

In [4]:
# calculate ngram frequency.
# n=1 is the most frequent of song_id
# n=2 is the most frequent of [song_id and next1_song_id], get the result from next2_song_id column
def getFreq(df, n=2):
    df_train = getTrainData(df, n)
    # calculate ngram frequency
    df_freq = (
        df_train.groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .value_counts(sort=True, normalize=True)
        .reset_index(name="freq")
    )

    # get the most frequent song_id, sort by freq
    df_freq = (
        df_freq.sort_values(
            ["song_id"] + [f"next{i}_song_id" for i in range(1, n)] + ["freq"],
            ascending=False,
        )
        .groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .head(1)
    )
    return df_freq

In [5]:
# get testX dataset
def getTestX(df, n=2):
    trainX = pd.DataFrame()
    # if n == 3:, get listening_order : 18, 19, 20
    for i in range(n, 0, -1):
        _ = df.query(f"listening_order == {20-i+1}")[
            ["session_id", "song_id"]
        ].set_index("session_id")
        if trainX.empty:
            trainX = _
        else:
            trainX = trainX.join(_, lsuffix=f"_{n-i-1}")

    trainX = trainX.rename(columns={trainX.columns[-1]: f"song_id_{n-1}"})

    # trainX keep session_id
    trainX.reset_index(inplace=True)
    trainX.rename_axis("session_id", axis=1, inplace=True)
    return trainX


testX = getTestX(test_source, n=2)
testX.head(5)

session_id,session_id,song_id_0,song_id_1
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516


In [6]:
# predict the next song id by ngram frequency table
def predict(df, freqTable, n=2):
    df = pd.merge(
        df,
        freqTable,
        how="left",
        left_on=[f"song_id_{i}" for i in range(n)],
        right_on=["song_id"] + [f"next{_}_song_id" for _ in range(1, n)],
    )
    for i in range(1, n):
        del df[f"next{i}_song_id"]
    del df["song_id"]
    df.rename(columns={f"next{n}_song_id": f"next_song_id"}, inplace=True)
    df.sort_values("session_id")
    return df

In [7]:
# merge the predicted result and new way freq table
def merge2freq(df, freq_table):
    # offset=2 : session_id, freq, next_song_id
    df_offset = 3
    # freq_offset: next_song_id, freq
    freq_offset = 2

    # Delete the unneed column:freq
    if "freq" in df.columns:
        df = df.drop(columns=["freq"])
        df_offset -= 1
    if "freq" in freq_table.columns:
        freq_table = freq_table.drop(columns=["freq"])
        freq_offset -= 1

    fomr_count = df.shape[1] - df_offset  # 2gram : 5 -3 = 2
    latr_count = freq_table.shape[1] - freq_offset  # 1gram freq: 3 - 2 = 1

    left_on = [f"song_id_{i+(fomr_count-latr_count)}" for i in range(latr_count)]
    right_on = ["song_id"] + [f"next{i+1}_song_id" for i in range(latr_count - 1)]
    print("Left on:", left_on)
    print("Right on:", right_on)

    print("r:" + f"song_id_{i}" for i in range(latr_count))
    merged = pd.merge(
        df,
        freq_table,
        how="left",
        left_on=[f"song_id_{i+(fomr_count-latr_count)}" for i in range(latr_count)],
        right_on=["song_id"] + [f"next{i+1}_song_id" for i in range(latr_count - 1)],
    )

    freq_target_name = f"next{latr_count}_song_id"
    merged["next_song_id"].fillna(merged[freq_target_name], inplace=True)

    merged = merged.drop(columns=right_on + [freq_target_name])

    return merged

# experiments

In [8]:
# map song_id to song_index and set song_idx to song to save memory and speed up
meta_song["song_index"] = meta_song.index
train_source = train_source.merge(
    meta_song[["song_id", "song_index"]], on="song_id", how="left"
)
train_target = train_target.merge(
    meta_song[["song_id", "song_index"]], on="song_id", how="left"
)
test_source = test_source.merge(
    meta_song[["song_id", "song_index"]], on="song_id", how="left"
)

train_source["song_id"] = train_source["song_index"]
train_target["song_id"] = train_target["song_index"]
test_source["song_id"] = test_source["song_index"]

del train_source["song_index"]
del train_target["song_index"]
del test_source["song_index"]

In [9]:
trainX = pd.concat([train_source, test_source], axis=0, ignore_index=True)

In [10]:
n = 1
testX = getTestX(test_source, n)
freq1 = getFreq(trainX, n)
freq1
result1 = predict(testX, freq1, n)
result1.head()

,session_id,song_id_0,next_song_id,freq
0,598,320485,320485.0,0.049930
1,1039,82103,245916.0,0.357143
2,1199,82260,82260.0,0.097738
3,1489,163516,58880.0,0.500000
4,1868,736058,798178.0,0.069952


In [11]:
result1.sort_values(["song_id_0", "session_id"], inplace=True)
result1

,session_id,song_id_0,next_song_id,freq
64449,294268,21,21.0,0.049062
139062,301108,21,21.0,0.049062
117995,390969,21,21.0,0.049062
18743,541198,21,21.0,0.049062
25912,116369,39,270774.0,0.165517
...,...,...,...,...
53933,544689,1030708,736280.0,0.170886
135082,577739,1030708,736280.0,0.170886
72807,624199,1030708,736280.0,0.170886
112178,240878,1030709,915034.0,0.046632


In [12]:
n = 2
testX = getTestX(test_source, n)
freq2 = getFreq(trainX, n)
result2 = predict(testX, freq2, n)
freq2

,song_id,next1_song_id,next2_song_id,freq
10095227,1030711,967508.0,561432.0,1.0
10095226,1030711,967484.0,609494.0,1.0
10095225,1030711,946599.0,125355.0,1.0
10095224,1030711,936567.0,1023129.0,1.0
10095223,1030711,643112.0,6835.0,1.0
...,...,...,...,...
4,3,401307.0,318166.0,1.0
3,3,200709.0,1021838.0,1.0
2,3,87846.0,897082.0,1.0
1,1,1014814.0,799780.0,1.0


In [13]:
freq2.sort_values(["song_id", "next1_song_id"], inplace=True)
freq2

,song_id,next1_song_id,next2_song_id,freq
0,1,488417.0,697989.0,1.0
1,1,1014814.0,799780.0,1.0
2,3,87846.0,897082.0,1.0
3,3,200709.0,1021838.0,1.0
4,3,401307.0,318166.0,1.0
...,...,...,...,...
10095223,1030711,643112.0,6835.0,1.0
10095224,1030711,936567.0,1023129.0,1.0
10095225,1030711,946599.0,125355.0,1.0
10095226,1030711,967484.0,609494.0,1.0


In [14]:
n = 2
testX = getTestX(test_source, n)
freq2 = getFreq(trainX, n)
result2 = predict(testX, freq2, n)
# result2.head()

# 2 1
result2_1 = merge2freq(result2, freq1)
result2_1.head()

Left on: ['song_id_1']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7f2fde3b4970>


,session_id,song_id_0,song_id_1,next_song_id
0,598,487545,320485,801419.0
1,1039,388680,82103,245916.0
2,1199,187475,82260,82260.0
3,1489,672342,163516,58880.0
4,1868,798178,736058,1001782.0


In [15]:
n = 3
testX = getTestX(test_source, n)
freq3 = getFreq(trainX, n)
result3 = predict(testX, freq3, n)
# result3.head()


# 3 2
result3_2 = merge2freq(result3, freq2)
# [3,2] 1
result3_2_1 = merge2freq(result3_2, freq1)
# [3,2,1] 20
result3_2_1["next_song_id"].fillna(result3_2_1["song_id_2"], inplace=True)
result = result3_2_1
result.head()

Left on: ['song_id_1', 'song_id_2']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7f2fde3b4f90>
Left on: ['song_id_1']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7f2fde3b4f90>


,session_id,song_id_0,song_id_1,song_id_2,next_song_id
0,598,309466,487545,320485,801419.0
1,1039,810606,388680,82103,388680.0
2,1199,524289,187475,82260,419795.0
3,1489,674238,672342,163516,705778.0
4,1868,935208,798178,736058,710126.0


In [16]:
n = 5
testX = getTestX(test_source, n)
freq5 = getFreq(trainX, n)
result5 = predict(testX, freq5, n)

# 5 2
result5_2 = merge2freq(result5, freq2)

# [5,2] 20
result5_2["next_song_id"].fillna(result5_2["song_id_4"], inplace=True)
result = result5_2
result.head()

Left on: ['song_id_3', 'song_id_4']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7f2fde3b4ac0>


,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,next_song_id
0,598,442405,834888,309466,487545,320485,801419.0
1,1039,229992,252014,810606,388680,82103,82103.0
2,1199,524289,976710,524289,187475,82260,82260.0
3,1489,847611,642783,674238,672342,163516,163516.0
4,1868,742718,856473,935208,798178,736058,710126.0


In [17]:
# calculate ngram frequency.
# n=1 is the most frequent of song_id
# n=2 is the most frequent of [song_id and next1_song_id], get the result from next2_song_id column
def getFreq2(df, n=2, predict=1):
    # get df's n+predict numbers of song id
    df_train = getTrainData(df, n + predict - 1)
    # calculate ngram frequency
    df_freq = (
        df_train.groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .value_counts(sort=True, normalize=True)
        .reset_index(name="freq")
    )

    # get the most frequent song_id, sort by freq
    df_freq = (
        df_freq.sort_values(
            ["song_id"] + [f"next{i}_song_id" for i in range(1, n)] + ["freq"],
            ascending=False,
        )
        .groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .head(1)
    )
    return df_freq


# predict the next song id by ngram frequency table
def predict2(df, freqTable, n=2, target=2):
    df = pd.merge(
        df,
        freqTable,
        how="left",
        left_on=[f"song_id_{i}" for i in range(n)],
        right_on=["song_id"] + [f"next{_}_song_id" for _ in range(1, n)],
    )
    for i in range(1, n):
        del df[f"next{i}_song_id"]
    del df["song_id"]
    for i in range(target):
        df.rename(columns={f"next{n+i}_song_id": f"top{i+1}"}, inplace=True)
    df.sort_values("session_id")
    return df

In [18]:
n = 3
target = 5
testX = getTestX(test_source, n)
freq3_5 = getFreq2(trainX, n, target)
result3_5 = predict2(testX, freq3_5, n, target)

_ = result3.rename(columns={"next_song_id": "top1"})
result3_5_3 = result3_5.fillna(_)
result = result3_5_3

In [19]:
result3_5

,session_id,song_id_0,song_id_1,song_id_2,top1,top2,top3,top4,top5,freq
0,598,309466,487545,320485,NaN,NaN,NaN,NaN,NaN,NaN
1,1039,810606,388680,82103,NaN,NaN,NaN,NaN,NaN,NaN
2,1199,524289,187475,82260,NaN,NaN,NaN,NaN,NaN,NaN
3,1489,674238,672342,163516,NaN,NaN,NaN,NaN,NaN,NaN
4,1868,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.272727
...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535.0,1028535.0,1028535.0,1028535.0,1028535.0,0.988263
143060,317578,140463,508806,352697,NaN,NaN,NaN,NaN,NaN,NaN
143061,712209,539300,943003,831678,NaN,NaN,NaN,NaN,NaN,NaN
143062,713779,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.978723


In [20]:
merged = pd.merge(
    result,
    freq2,
    how="left",
    left_on=["song_id_1", "song_id_2"],
    right_on=["song_id", "next1_song_id"],
)

merged["top1"].fillna(merged["next2_song_id"], inplace=True)
merged["top1"].fillna(merged["song_id_2"], inplace=True)
merged = merged.drop(
    columns=["freq_x", "song_id", "next1_song_id", "next2_song_id", "freq_y"]
)
result = merged
result

,session_id,song_id_0,song_id_1,song_id_2,top1,top2,top3,top4,top5
0,598,309466,487545,320485,801419.0,NaN,NaN,NaN,NaN
1,1039,810606,388680,82103,82103.0,NaN,NaN,NaN,NaN
2,1199,524289,187475,82260,82260.0,NaN,NaN,NaN,NaN
3,1489,674238,672342,163516,163516.0,NaN,NaN,NaN,NaN
4,1868,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0
...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535.0,1028535.0,1028535.0,1028535.0,1028535.0
143060,317578,140463,508806,352697,352697.0,NaN,NaN,NaN,NaN
143061,712209,539300,943003,831678,831678.0,NaN,NaN,NaN,NaN
143062,713779,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0


In [21]:
result["top2"].fillna(result["top1"], inplace=True)
result["top3"].fillna(result["top1"], inplace=True)
result["top4"].fillna(result["top1"], inplace=True)
result["top5"].fillna(result["top1"], inplace=True)

In [22]:
n = 5
target = 3
testX = getTestX(test_source, n)
freq5_3 = getFreq2(trainX, n, target)
result5_3 = predict2(testX, freq5_3, n, target)


result5_3

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,freq
0,598,442405,834888,309466,487545,320485,NaN,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,NaN,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,NaN,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,0.600000
...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,1028535.0,1028535.0,0.992925
143060,317578,467984,610999,140463,508806,352697,NaN,NaN,NaN,NaN
143061,712209,578489,16604,539300,943003,831678,NaN,NaN,NaN,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,0.997831


In [23]:
nan_count = result["next_song_id"].isna().sum()
print(nan_count / result.shape[0])

KeyError: 'next_song_id'

### Drop testX input

In [ ]:
n = 3
drop_cols = [f"song_id_{i}" for i in range(n)]
result.drop(columns=drop_cols, inplace=True)

result.rename(columns={"next_song_id": "top1"}, inplace=True)

### Duplicate Top1 to top2, top3, ... , top5

In [ ]:
for i in range(2, 6):
    result[f"top{i}"] = result["top1"]
result.head()

,session_id,top1,top2,top3,top4,top5
0,8,715921.0,715921.0,715921.0,715921.0,715921.0
1,9,950108.0,950108.0,950108.0,950108.0,950108.0
2,18,571377.0,571377.0,571377.0,571377.0,571377.0
3,19,652741.0,652741.0,652741.0,652741.0,652741.0
4,28,107386.0,107386.0,107386.0,107386.0,107386.0


### Ramdom song id to top2~top5

### Ramdom pick 22, 23, 24, 25

In [ ]:
res = result.copy()
for i in range(2, 6):
    _sample = meta_song.sample(result.shape[0], replace=True)["song_id"].reset_index(
        drop=True
    )
    res[f"top{i}"] = pd.Series(_sample)
res.head()

,session_id,top1,top2,top3,top4,top5
0,8,715921.0,b2f773ab852031283c39de28c8594773,3a2f0200de79407d41e61539a4cecce0,be74c01b2f27bef2e0aea3bdf3df32f9,d82125c335c2fa41736189af98201f6a
1,9,950108.0,b94ba1abf2b14b4ad39fa1dfd644f698,a0093873b76c5c40c46ea10b2137abff,6b13ac51a21f1953f0b7970bd65e9aa6,fe2f38596f24c3854f450b412ad9c4da
2,18,571377.0,460c329e739ad79468e785f959ceba2e,0be18bde9ca74866d050615a447f5e59,b22b6c157b81ea1e672acfd69dcaaf74,e5af8bd0c1d53c13c32162d522fcbc62
3,19,652741.0,98ee572a927f539b94ee7b1b4245b454,470b8bed7cb398cbdb3c705e83b344bf,002e690dd874706610ca874e7d5c1048,cd9627a6c417359b6c441c60f9a56bda
4,28,107386.0,64a8ef7cd15951b66c5bfb1639c5597d,feb85456c237d618b0d1d8e24fcd9c34,fa6afb41c95d9272e86e7c933636fa55,f2f39a8a409724f6f587c23a3939f4ee


### Output to csv file

In [ ]:
result.to_csv("results/result.csv", index=False)

In [ ]:
result.head()

,session_id,top0,top1,top2,top3,top4
0,598,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943
1,1039,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f
2,1199,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3
3,1489,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d
4,1868,269ef1b3c7933d8de1d8dcb834630a4c,23cb1d20ca90b3574d6f9537c9198525,1943cae8d1ded4ea75126e5da5b87916,c534b7ad0d186ee663f11d29642d692c,1ae353d3f12ec67cbf67ce184b017cd3


In [ ]:
df = pd.read_csv("~/Downloads/result.csv")

# Display the first few rows of the DataFrame
df.head()

,session_id,top1,top2,top3,top4,top5
0,598,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943
1,1039,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f,308f5f2b339b3ff74c9b60b49832d27f
2,1199,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3
3,1489,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d,4c19e9de0a90902ccd3b55374942fc5d
4,1868,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c


In [2]:
import numpy as np


def numpy_to_csv(filename, data_array, column_names):
    # Check if the number of columns in data_array matches the length of column_names
    if data_array.shape[1] != len(column_names):
        raise ValueError(
            "Number of columns in data_array must match the length of column_names."
        )

    # Create a list with column names and data_array
    combined_data = [column_names] + data_array.tolist()

    # Save the combined list to a CSV file
    np.savetxt(filename, combined_data, delimiter=",", fmt="%s", header="", comments="")

    print(f"CSV file '{filename}' saved successfully.")


# Example usage:
# Create a sample NumPy array (replace this with your data)
my_array = np.random.random((10, 6))

# Define column names
my_column_names = ["Column1", "Column2", "Column3", "Column4", "Column5", "Column6"]

# Call the function to save the array to CSV with column names
numpy_to_csv("output.csv", my_array, my_column_names)

CSV file 'output.csv' saved successfully.
